In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

# Carga y Exploración Inicial de Datos

## Carga del Dataset

Se procede a cargar el dataset desde el archivo CSV ubicado en la carpeta `data/`. Este archivo contiene los datos necesarios para la Etapa 1 del proyecto.

## Análisis Estadístico Descriptivo

Se utiliza la función `describe()` para obtener un resumen estadístico de las variables numéricas presentes en el dataset. Esta función proporciona métricas clave como:

- **count**: Cantidad de valores no nulos en cada columna
- **mean**: Promedio de los valores
- **std**: Desviación estándar (dispersión de los datos)
- **min/max**: Valores mínimo y máximo
- **25%, 50%, 75%**: Percentiles que indican la distribución de los datos

Este análisis permite identificar rápidamente patrones, valores atípicos y la distribución general de las variables numéricas del dataset.

In [2]:
data_raw = pd.read_csv("../data/Datos_Etapa-1.csv")
data_raw.describe()
data_raw.head()

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0,1,Night


## Identificación de Valores Faltantes

Se analiza la presencia de valores nulos en el dataset mediante `isna().sum()`, que cuenta la cantidad de valores faltantes por columna.

Este paso permite identificar qué variables tienen datos incompletos y determinar la estrategia de limpieza apropiada (imputación o eliminación).

In [3]:
data_raw.isna().sum()

season         0
weekday        0
weathersit     0
temp           0
atemp          0
hum            0
windspeed      0
cnt            0
time_of_day    0
dtype: int64

## Detección de Registros Duplicados

Se verifica la cantidad de filas duplicadas en el dataset usando `duplicated().sum()`. Esta función identifica registros que son idénticos en todas sus columnas.

Detectar duplicados es esencial para asegurar la calidad de los datos y evitar sesgos en el análisis posterior.

In [4]:
data_raw.duplicated().sum()

np.int64(42)

## Eliminación de Duplicados

Se procede a eliminar todas las filas duplicadas del dataset usando `drop_duplicates()`. Posteriormente, se verifica con `duplicated().sum()` que la eliminación fue exitosa.

Este proceso asegura que cada registro sea único, evitando redundancia y posibles distorsiones en el análisis de datos.

In [5]:
data = data_raw.drop_duplicates()
data.duplicated().sum()

np.int64(0)

## Codificación de Variables Categóricas

Se aplica **One-Hot Encoding** a las variables categóricas (`season`, `weathersit`, `time_of_day`), transformándolas en variables binarias (0 o 1).

In [6]:
data_encoded = data.copy()
data_encoded = pd.get_dummies(data_encoded, columns=['season', 'weathersit', 'time_of_day'], drop_first=True)
data_encoded.head()

,weekday,temp,atemp,hum,windspeed,cnt,season_Spring,season_Summer,season_Winter,weathersit_Heavy Rain,weathersit_Light Rain,weathersit_Mist,time_of_day_Morning,time_of_day_Night
0,6,3.28,3.0014,0.81,0.0,16,False,False,True,False,False,False,False,True
1,6,2.34,1.9982,0.80,0.0,40,False,False,True,False,False,False,False,True
2,6,2.34,1.9982,0.80,0.0,32,False,False,True,False,False,False,False,True
3,6,3.28,3.0014,0.75,0.0,13,False,False,True,False,False,False,False,True
4,6,3.28,3.0014,0.75,0.0,1,False,False,True,False,False,False,False,True


## Preparación de Datos para Entrenamiento

Se separan las variables predictoras (features) de la variable objetivo:

- **X**: Contiene todas las variables independientes (features) excepto `cnt`
- **y**: Contiene la variable objetivo `cnt` (demanda de bicicletas)

Posteriormente, se divide el dataset en conjuntos de entrenamiento y prueba usando `train_test_split`:

- **80%** de los datos para entrenamiento (`x_train`, `y_train`)
- **20%** de los datos para prueba (`x_test`, `y_test`)
- `random_state=77` asegura reproducibilidad de los resultados

Esta división permite evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento.

In [7]:
x = data_encoded.drop(['cnt'], axis="columns")
y = data_encoded['cnt']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=77)

## Creación de Features Polinomiales (Grado 2)

Se generan características polinomiales de grado 2 mediante `PolynomialFeatures`. Esto permite capturar relaciones no lineales entre variables, creando términos cuadráticos e interacciones entre features.

Posteriormente, se normaliza el dataset transformado usando **RobustScaler** para mantener todas las variables en una escala comparable y preservar la estabilidad numérica del modelo.

In [8]:
pol_2 = PolynomialFeatures(degree=2)
x_train_pol2 = pol_2.fit_transform(x_train)

pol_2_scaler = RobustScaler()
x_train_pol2 = pol_2_scaler.fit_transform(x_train_pol2)

## Entrenamiento del Modelo de Regresión Polinomial (Grado 2)

Se entrena un modelo de **Regresión Lineal** utilizando las características polinomiales de grado 2. El modelo aprende los pesos óptimos que minimizan el error entre las predicciones y los valores reales de demanda en el conjunto de entrenamiento.

In [9]:
reg_polinomial2 = LinearRegression().fit(x_train_pol2, y_train)

## Transformación del Conjunto de Prueba

Se aplican las mismas transformaciones al conjunto de prueba:

- Se generan características polinomiales de grado 2 usando el transformador ajustado previamente
- Se normaliza con el escalador ya entrenado

In [10]:
x_test_pol2 = pol_2.transform(x_test)
x_test_pol2 = pol_2_scaler.transform(x_test_pol2)

## Evaluación del Modelo Polinomial (Grado 2)

Se realizan predicciones sobre el conjunto de prueba y se evalúa el rendimiento del modelo mediante tres métricas:

- **RMSE** (Root Mean Squared Error): Mide el error promedio de las predicciones, penalizando más los errores grandes
- **MAE** (Mean Absolute Error): Representa el error absoluto promedio en las mismas unidades que la variable objetivo
- **R²** (Coeficiente de determinación): Indica qué porcentaje de la variabilidad de la demanda es explicado por el modelo (valores cercanos a 1 indican mejor ajuste)

In [11]:
y_pred_pol2 = reg_polinomial2.predict(x_test_pol2)

print(f'------ Modelo de regresión polinomial grado 2 ----')
print(f"RMSE: {root_mean_squared_error(y_test, y_pred_pol2):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_pol2):.2f}")
print(f'R²: {r2_score(y_test, y_pred_pol2):.2f}')


------ Modelo de regresión polinomial grado 2 ----
RMSE: 135.96
MAE: 99.08
R²: 0.45


## Creación de Features Polinomiales (Grado 3)

Se generan características polinomiales de grado 3 mediante `PolynomialFeatures`. Esto permite capturar relaciones no lineales más complejas entre variables, creando términos cúbicos e interacciones de mayor orden.

Posteriormente, se normaliza el dataset transformado usando **RobustScaler** para mantener todas las variables en una escala comparable y preservar la estabilidad numérica del modelo.

In [12]:
pol_3 = PolynomialFeatures(degree=3)
x_train_pol3 = pol_3.fit_transform(x_train)

pol_3_scaler = RobustScaler()
x_train_pol3 = pol_3_scaler.fit_transform(x_train_pol3)

## Entrenamiento del Modelo de Regresión Polinomial (Grado 3)

Se entrena un modelo de **Regresión Lineal** utilizando las características polinomiales de grado 3. El modelo aprende los pesos óptimos que minimizan el error entre las predicciones y los valores reales de demanda en el conjunto de entrenamiento.

In [13]:
reg_polinomial3 = LinearRegression().fit(x_train_pol3, y_train)

## Transformación del Conjunto de Prueba

Se aplican las mismas transformaciones al conjunto de prueba:

- Se generan características polinomiales de grado 3 usando el transformador ajustado previamente
- Se normaliza con el escalador ya entrenado

In [14]:
x_test_pol3 = pol_3.transform(x_test)
x_test_pol3 = pol_3_scaler.transform(x_test_pol3)

## Evaluación del Modelo Polinomial (Grado 3)

Se realizan predicciones sobre el conjunto de prueba y se evalúa el rendimiento del modelo mediante tres métricas:

- **RMSE** (Root Mean Squared Error): Mide el error promedio de las predicciones, penalizando más los errores grandes
- **MAE** (Mean Absolute Error): Representa el error absoluto promedio en las mismas unidades que la variable objetivo
- **R²** (Coeficiente de determinación): Indica qué porcentaje de la variabilidad de la demanda es explicado por el modelo (valores cercanos a 1 indican mejor ajuste)

In [16]:
y_pred_pol3 = reg_polinomial3.predict(x_test_pol3)

print(f'------ Modelo de regresión polinomial grado 3 ----')
print(f"RMSE: {root_mean_squared_error(y_test, y_pred_pol3):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_pol3):.2f}")
print(f'R²: {r2_score(y_test, y_pred_pol3):.2f}')

------ Modelo de regresión polinomial grado 3 ----
RMSE: 132.31
MAE: 96.12
R²: 0.48


## Selección del Modelo Óptimo

Se selecciona el **modelo de grado 3** como el más adecuado para predecir la demanda de bicicletas.

**Justificación:**

El modelo de grado 3 presenta un **RMSE de 132.31**, menor que el de grado 2 (RMSE: 135.96). Esta mejora indica que:

- Realiza predicciones más precisas
- Captura mejor los patrones en los datos
- Ofrece un buen equilibrio entre rendimiento y complejidad

El modelo de grado 3 se utiliza como modelo final por su mejor desempeño predictivo.